# Connecting to an API/Pulling in the Data and Cleaning/Formatting

In [35]:
import requests
import json
import pandas as pd
import numpy as np

**The following can be skipped by using the booksAPI.csv file**

In [2]:
books = pd.read_csv('books.csv', error_bad_lines=False)

# There is a space in the column num_pages
books.rename(columns=lambda x: x.strip(), inplace=True)

# list of isbn numbers to access
isbn = books['isbn13'].to_list()

# Used to pull data from API in chunks of 500  
isbn1 = isbn[:500] 
isbn2 = isbn[500:1000] 
isbn3 = isbn[1000:1500] 
isbn4 = isbn[1500:2000] 
isbn5 = isbn[2000:2500] 
isbn6 = isbn[2500:3000] 
isbn7 = isbn[3000:3500] 
isbn8 = isbn[3500:4000] 
isbn9 = isbn[4000:4500]
isbn10 = isbn[4500:5000]
isbn11 = isbn[5000:5500] 
isbn12 = isbn[5500:6000] 
isbn13 = isbn[6000:6500]
isbn14 = isbn[6500:7000] 
isbn15 = isbn[7000:7500] 
isbn16 = isbn[7500:8000]
isbn17 = isbn[8000:8500] 
isbn18 = isbn[8500:9000] 
isbn19 = isbn[9000:9500] 
isbn20 = isbn[9500:10000] 
isbn21 = isbn[10000:10500] 
isbn22 = isbn[10500:11000]
isbn23 = isbn[11000:]

keys = ['info_url', 'bib_key', 'preview_url', 'thumbnail_url', 'preview']

lst = []
# For each channel, we access its information through its API
for book in isbn23:
    url = 'https://openlibrary.org/api/books?bibkeys=ISBN:{}&jscmd=details&format=json'.format(book)
    JSONContent = requests.request('GET', url)
    data = json.loads(JSONContent.text)

    for key in keys:
        try:
            lst.append(data['ISBN:{}'.format(book)][key])
        except KeyError:
            lst.append('NaN')

a = np.array(lst)
b = a.reshape(len(isbn23), len(keys))
df = pd.DataFrame(b, columns = keys)


details = ['isbn_13', 'title', 'latest_revision', 'revision', 'edition_name', 'genres', 'languages', 'subjects', 
           'publish_country', 'by_statement', 'revision', 'other_titles', 'publishers', 'last_modified', 'authors', 
           'publish_places', 'created', 'notes', 'number_of_pages', 'publish_date', 'series', 
           'physical_format', 'description']

lst = []
# For each channel, we access its information through its API
for book in isbn23:
    url = 'https://openlibrary.org/api/books?bibkeys=ISBN:{}&jscmd=details&format=json'.format(book)
    JSONContent = requests.request('GET', url)
    data = json.loads(JSONContent.text)

    for key in details:
        try:
            lst.append(data['ISBN:{}'.format(book)]['details'][key])
        except KeyError:
            lst.append('NaN')

a = np.array(lst)
b = a.reshape(len(isbn23), len(details))
df2 = pd.DataFrame(b, columns = details)

# Used to create a csv file with the API data so this process did not have to be repeated
new2 = pd.concat([df, df2], axis=1) 
new2.to_csv('booksAPI_2.csv', sep=',', index=False)
first = pd.read_csv('booksAPI.csv')
second = pd.read_csv('booksAPI_2.csv')
a = pd.concat([first, second])
a.to_csv('booksAPI.csv', sep=',', index=False)

b'Skipping line 3350: expected 12 fields, saw 13\nSkipping line 4704: expected 12 fields, saw 13\nSkipping line 5879: expected 12 fields, saw 13\nSkipping line 8981: expected 12 fields, saw 13\n'


In [36]:
# The CSV file of the API data
df = pd.read_csv('booksAPI.csv')

In [37]:
df.columns

Index(['info_url', 'bib_key', 'preview_url', 'thumbnail_url', 'preview',
       'isbn_13', 'title', 'latest_revision', 'revision', 'edition_name',
       'genres', 'languages', 'subjects', 'publish_country', 'by_statement',
       'revision.1', 'other_titles', 'publishers', 'last_modified', 'authors',
       'publish_places', 'created', 'notes', 'number_of_pages', 'publish_date',
       'series', 'physical_format', 'description'],
      dtype='object')

In [38]:
df.describe()

,latest_revision,revision,revision.1,number_of_pages
count,10945.000000,10945.000000,10945.000000,9460.000000
mean,8.356510,8.356510,8.356510,340.399471
std,3.464957,3.464957,3.464957,226.567029
min,1.000000,1.000000,1.000000,1.000000
25%,6.000000,6.000000,6.000000,202.000000
50%,8.000000,8.000000,8.000000,302.500000
75%,10.000000,10.000000,10.000000,416.000000
max,68.000000,68.000000,68.000000,4736.000000


In [39]:
# latest_revision, revision, and revision.1 are the same values. I will remove two of them.
df.drop(columns=['latest_revision', 'revision.1'], axis=1, inplace=True)

In [40]:
# I want to number_of_pages to num_pages and physical format to format.
df.rename(columns={'number_of_pages': 'pages', 'physical_format': 'format'}, inplace=True)

In [41]:
# Check for missing data
df.isnull().sum()

info_url             153
bib_key              153
preview_url          153
thumbnail_url       1791
preview              153
isbn_13             4551
title                154
revision             153
edition_name        7652
genres              8904
languages           1327
subjects            3511
publish_country     5400
by_statement        5555
other_titles       10273
publishers           191
last_modified        153
authors              932
publish_places      5181
created              184
notes               7315
pages               1638
publish_date         315
series              8916
format              7419
description        10025
dtype: int64

In [42]:
# Many of the variables are missing 153 values. I am going to delete any observations that do not have a title.
df.dropna(subset=['title'], inplace=True)

In [43]:
# Now I want to format some of the data.
# Extract the author from 'authors'
df['author'] = df['authors'].apply(lambda x: np.nan if pd.isnull(x) else x.split(':')[1].split(',')[0].replace("'",''))

# The bib_key variable is the ISBN number. I want to remove "ISBN:" from the values
df['isbn13'] = df['bib_key'].apply(lambda x: int(x.split(':')[1]))

# Extract date from created.
df['created'] = df['created'].apply(lambda x: np.nan if pd.isnull(x) 
                                      else x.split("'value':")[-1].replace("'", '').replace('}', ''))

# Extract date last modified
df['modified'] = df['last_modified'].apply(lambda x: np.nan if pd.isnull(x) 
                                      else x.split("'value':")[-1].replace("'", '').replace('}', ''))

# Extract the book description 
df['description'] = df['description'].apply(lambda x: np.nan if pd.isnull(x) 
                                     else x.split("'value':")[-1].replace('"','').replace("'",'').replace('}','').lstrip())

# Extract the publishers
df['publishers'] = df['publishers'].apply(lambda x: np.nan if pd.isnull(x) 
                                          else x.replace('[','').replace(']','').replace("'",''))

# Exract genre
df['genres'] = df['genres'].apply(lambda x: np.nan if pd.isnull(x) 
                                          else x.replace('[','').replace(']','').replace("'",'').replace('.','').lower())

# Extract language
df['languages'] = df['languages'].apply(lambda x: np.nan if pd.isnull(x) 
                                          else x.split('/')[-1].replace('}','').replace(']','').replace("'",''))

# Format book format
df['format'] = df['format'].apply(lambda x: np.nan if pd.isnull(x) else x.lower())

# Format notes
df['notes'] = df['notes'].apply(lambda x: np.nan if pd.isnull(x) 
                                     else x.split("'value':")[-1].replace('"','').replace("'",'').replace('}','').lstrip())

# Format subjects
df['subjects'] = df['subjects'].apply(lambda x: np.nan if pd.isnull(x) 
                                          else x.strip('[]').replace("'",''))

In [44]:
# Now, I can drop authors, isbn_13, bib_key, and last_modified. I also want to remove other_titles, edition_name, 
# publish_places, series since most of the values are missing and by_statement as it contains the author name.  
df.drop(columns=['authors', 'isbn_13', 'bib_key','last_modified', 'publish_places', 'series', 'other_titles', 
                 'edition_name', 'by_statement'], axis=1, inplace=True)

In [45]:
# I want to see how many unique values for each variable
for var in df.columns:
    num = df[var].nunique()
    print(var, num)

info_url 10903
preview_url 10903
thumbnail_url 9255
preview 4
title 10176
revision 38
genres 168
languages 20
subjects 6941
publish_country 77
publishers 2563
created 3499
notes 2891
pages 966
publish_date 2288
format 35
description 1062
author 4002
isbn13 10944
modified 10285


In [46]:
# convert created and modified from a string to datetime
import datetime
df['created'] = pd.to_datetime(df['created'], format='%Y/%m/%d', errors='coerce')
df['modified'] = pd.to_datetime(df['modified'], format='%Y/%m/%d', errors='coerce')

# Extract the year from publish_date
df['year'] = df['publish_date'].apply(lambda x: np.nan if pd.isnull(x) else re.findall('\d{4}', x))
df['year'] = df['year'].apply(lambda x: str(x).strip('[]').replace("'",''))
df['year'] = pd.to_numeric(df['year'], errors='coerce')# I want to make it an integer. It is a float

# Remove publish_date
df.drop(columns=['publish_date'], axis=1, inplace=True)

In [47]:
# I want to look closer into the format variable
df['format'].unique()

array(['paperback', 'trade paperback', 'hardcover', nan, 'audio cd',
       'mass market paperback', 'school & library binding',
       'unknown binding', 'comic', 'texte imprimé', 'ebook (kindle)',
       'leather bound', 'mp3 cd', 'electronic resource', 'board book',
       'softcover', 'library binding', 'digital audio', 'large print',
       'pamphlet', 'audio cassette', 'turtleback', 'cd-rom',
       'paperback; hardcover', 'spiral-bound', 'multimedia', 'erotica',
       'three hardcover with slipcase, hardcover',
       '[electronic resource] /', 'flexibound', 'oversize paperback',
       'ebook', 'hard cover', 'audio book', '[electronic resource]',
       'hardcover (gibraltar library binding)'], dtype=object)

In [48]:
# I want to narrow down the format types to paperback, audio, hardcover, ebook, and unknown
df['format'].replace(['trade paperback', 'softcover', 'mass market paperback', 'paperback; hardcover', 'flexibound',
                       'oversize paperback'], 'paperback', inplace=True)

df['format'].replace(['audio cd', 'mp3 cd', 'digital audio', 'audio cassette', 'audio book', ], 'audio', inplace=True)
                      
    
df['format'].replace(['school & library binding', 'leather bound', "hardcover'", 'three hardcover with slipcase, hardcover', 
                       'hardcover (gibraltar library binding)', 'hard cover', 'turtleback', 'board book', 'library binding'], 
                     'hardcover', inplace=True)

df['format'].replace(['ebook (kindle)', '[electronic resource] /', 'electronic resource', 
                      '[electronic resource]', 'cd-rom', 'multimedia'], 'ebook', inplace=True)

df['format'].replace([np.nan, 'unknown binding', 'texte imprimé', 'large print', 'spiral-bound', 'erotica', 
                      'pamphlet', 'comic'], 'unknown', inplace=True)

In [49]:
# I want to remove text in parentheses from the titles and convert them to lowercase
df['title'] = df['title'].apply(lambda x: x.split(' (')[0])
df['title_lower'] = df['title'].apply(lambda x: x.replace(',', '').lower())

In [50]:
# Then, I will save the clean data as a CSV file
df.to_csv('clean_apibooks.csv', sep=',', index=False)